<center><h1>Informe Tarea 1.</h1>  
Tema:  Análisis de sentimientos en Tweets
  
Integrantes: Matías Rojas y María José Trujillo. </center>



<h2><b> Introducción. </b></h2>


El análisis de sentimientos en tweets pertenece a un área de investigación del procesamiento de lenguaje natural, con aplicaciones en cuanto al tratamiento   computacional   de   opiniones, sentimientos  y subjetividad  en  textos. Esta área  tiene diversas sub-áreas tales como la clasificación de textos binaria en negativos o positivos, en un rango de positividad o negatividad, entre otras. En esta tarea en particular, se analizará la intensidad de sentimientos en diversos textos que fueron obtenidos desde twitter, en específico se analizarán textos que representan sentimientos de enojo, alegría, miedo y tristeza, y se crearan modelos para clasificar en tres clases: bajo, medio y alto, que representan la intensidad del texto.

Formalmente, la task recibe como input un texto $w$, y devuelve como output una clase $c_{i}$ $\in$ {low,medium,high} y las métricas a utilizar para medir que tan acertado es el modelo serán auc, kappa y average.

Para poder entrenar el modelo se utilizará el procedimiento estandar al momento de entrenar modelos utilizando machine learning, es decir, tres conjuntos: training, validación y testing. El conjunto de training y validación es entregado por el cuerpo docente, son 4 conjuntos donde cada uno está asociado a cada sentimiento. Cada uno de estos conjuntos se  divide en un $66.6\%$ para entrenar y luego un $33.3\%$ para la validación. Finalmente, el conjunto de testing es un conjunto gold que será utilizado para medir la generalización del modelo y con esto se obtendrá el puntaje de la competencia.

En cuanto a los modelos, el enfoque de la tarea o más bien desafío será analizar que tan alta puede ser la efectividad de la predicción utilizando herramientas vistas hasta el momento en el curso, es decir, clasificadores. Para ello, se partirá con el baseline entregado por el cuerpo docente: 
<a href="https://github.com/dccuchile/CC6205/tree/master/assignments/assignment_1">Github,</a> y se probará con distintos clasificadores tales como SVM, NB, RandomForestClassifier, LogisticRegression, entre otros. Y distintas formas de vectorizar, tokenizando con herramientas como TweetTokenizer, utilización de n-gramas de palabras y caractéres, utilización de lexicons, entre otros. Finalmente, se realizará un cuadro comparativo de todos los métodos.


<h2><b>Trabajo relacionado.</h2></b>

Para poder analizar el estado del arte en esta área hay que recalcar que no se está estudiando la clasificación de un texto en positivo o negativo, ya que en esa área hay muchas técnicas ya conocidas que se comportan bien en cuanto a métricas. En este caso lo que se está haciendo es medir la intensidad que tiene un sentimiento en particular, lo que supone un gran desafío debido a que no hay mucho estudio en estos casos. 

En el <a href="https://github.com/dccuchile/CC6205/tree/master/assignments/assignment_1">paper</a> Emotion Intensities in Tweets escrito por Felipe Bravo y Saif M. Mohammad, se mencionan trabajos realizados en el área: 

Strapparava and Mihalcea (2007): Lo que se hizo fue que diferentes personas pudieran clasificar la intensidad de una noticia en una escala de 1 a 100 mediante una barras deslizantes mostradas en una interfaz web. El problema de este experimento es el sesgo que aparece al momento de comparar los puntajes asignados por distintas personas, debido a que para algunas personas la emoción en el texto puede ser muy intensa pero para otras no tanto. Y como tienen 100 alternativas de puntuación es muy difícil que coincidan en un rango pequeño y esto puede ocasionar error en el entrenamiento de los modelos, problemas de generalización, entre otros.

Kiritchenko y Mohammad (2017): Best-Worst Scaling es una tecnica desarrollada por Louviere en 1991 basada en investigaciones hechas en psicología matemática y psicofísica en 1960 por AnthonyA. J. Marley y Duncan Luce. 

Lo que se demostró a través de diversos experimentos es que  con BWS se abordan los desafíos de la puntuación directa y se producen puntajes de intensidad emocional más confiables en comparación a la asignación mediante barras deslizantes. Además, este será el primer conjunto de datos con puntajes emocionales para tweets.

De todas formas, se menciona que aún queda mucho por investigar e implementar en esta área.

<h2><b>Algoritmos y representaciones.</h2></b>

La idea será utilizar un código base donde lo único que irá variando a lo largo de los experimentos será el trozo de código con la función getclassifier(). La metodología para poder facilitar la revisión y reutilizar código será poner distintos trozos de códigos, cada uno asociado a un experimento en particular, luego el miembro del cuerpo docente puede ir variando los métodos. En general lo que se buscará antes de hacer algún submission será superar el promedio obtenido de las métricas en el baseline, pero haciendo un énfases en kappa para que el modelo no se comporte como aleatorio.



<h2><b>Experimentos</h2></b>

##Imports

In [95]:
# Imports necesarios para probar distintos clasificadores y tokenizaciones.

import pandas as pd
import shutil
import string
import re
import nltk
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, cohen_kappa_score, classification_report, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from nltk.tokenize import TweetTokenizer
from sklearn.linear_model import LogisticRegression
from nltk.sentiment.util import  mark_negation
from sklearn import svm
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.datasets import load_breast_cancer
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC  # support vector machine classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB  # naive bayes
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from nltk.corpus import opinion_lexicon  
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from nltk.stem.porter import PorterStemmer
import numpy as np 
porter_stemmer = PorterStemmer()


nltk.download('opinion_lexicon')

# Descarga del dataset 

train = {
    'anger': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/anger-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity']),
    'fear': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/fear-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity']),
    'joy': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/joy-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity']),
    'sadness': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/sadness-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'])
}

target = {
    'anger': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/anger-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'fear': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/fear-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'joy': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/joy-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'sadness': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/sadness-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE'])
}


[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


## Preprocesamiento:
Se ajusta el dataset para dejar todas las intensidades de sentimiento en la misma proporcion, es decir, se corta la clase medium para que tenga el mismo tamaño que high y low. De esta forma, no se sobresatura el clasificador con datos medium, evitando que sea más propenso a elegir esta intensidad.


In [96]:
def get_group_dist(group_name, train):
    print(group_name, "\n",
          train[group_name].groupby('sentiment_intensity').count())
from collections import Counter
from sklearn.utils import shuffle
nuevotrain= {}
import math
def subsampling(train):

  for key in train:
    a=Counter(train[key].sentiment_intensity)['low']
    b=Counter(train[key].sentiment_intensity)['high']
    promedio= math.floor((a+b)/2)
    medium= shuffle(train[key][train[key].sentiment_intensity == 'medium'][1:])[0:promedio].copy()
    high= train[key][train[key].sentiment_intensity == 'high'].copy()
    low= train[key][train[key].sentiment_intensity == 'low'].copy()
    data=low.append(high).copy()
    data1=data.append(medium)
    nuevotrain[key]=data1.copy()
 
subsampling(train)

for key in nuevotrain:
  get_group_dist(key, nuevotrain)


anger 
                       id  tweet  class
sentiment_intensity                   
high                 163    163    163
low                  161    161    161
medium               162    162    162
fear 
                       id  tweet  class
sentiment_intensity                   
high                 270    270    270
low                  288    288    288
medium               279    279    279
joy 
                       id  tweet  class
sentiment_intensity                   
high                 195    195    195
low                  219    219    219
medium               207    207    207
sadness 
                       id  tweet  class
sentiment_intensity                   
high                 197    197    197
low                  210    210    210
medium               203    203    203


## Metricas y Split


In [0]:
# Definición de la métrica de evaluación auc

def auc(test_set, predicted_set):
    high_predicted = np.array([prediction[2] for prediction in predicted_set])
    medium_predicted = np.array(
        [prediction[1] for prediction in predicted_set])
    low_predicted = np.array([prediction[0] for prediction in predicted_set])

    high_test = np.where(test_set == 'high', 1.0, 0.0)
    medium_test = np.where(test_set == 'medium', 1.0, 0.0)
    low_test = np.where(test_set == 'low', 1.0, 0.0)

    auc_high = roc_auc_score(high_test, high_predicted)
    auc_med = roc_auc_score(medium_test, medium_predicted)
    auc_low = roc_auc_score(low_test, low_predicted)

    auc_w = (low_test.sum() * auc_low + medium_test.sum() * auc_med +
             high_test.sum() * auc_high) / (
                 low_test.sum() + medium_test.sum() + high_test.sum())
    return auc_w 

def split_dataset(dataset):
    # Dividir el dataset en train set y test set
    X_train, X_test, y_train, y_test = train_test_split(
        dataset.tweet,
        dataset.sentiment_intensity,
        shuffle=True,
        test_size=0.3)
    return X_train, X_test, y_train, y_test

##Tipos de experimentos realizados


###Experimento 0:
Baseline equipo docente

In [0]:
def experimento_0():
    vectorizer = CountVectorizer()
    naive_bayes = MultinomialNB()
    text_clf = Pipeline([('vect', vectorizer), ('clf', naive_bayes)])
    return text_clf
    
    
    

###Experimento 1:
Consiste en mantener el clasificador utilizado (NaiveBayes) en el baseline pero variando el vectorizador. Para ello se realizaron diversas pruebas obteniendo como mejor configuración la utilización de TweetTokenizer, mark_negation y un análisis de n-gramas de palabras.
Se utiliza ngram_range(1,4) como se especifica en el paper "Emotion Intensities in Tweets" por Saif M. Mohammad y Felipe Bravo-Marquez


Se utiliza CountVectorizer como vectorizador y LogisticRegression. Se le agregan distintos paramétros para optimizar la classificación. A grandes rasgos estos son: 

TweetTokenizer: Tokenizador especial para tweets, clasifica emojis, borra las menciones (strip_handles) y hace downcase (preserve_case=False) 

CountVectorizer: Es el vectorizador que pasa el texto a BOW, también recibe varios parámetros que optimizan este cambio. Analyzer es como se tokenizaran los textos, por ejemplo, si este parámetro se encuentra en como 'word', se tokenizan palabras. Si es tipo 'char' se tokenizan caracteres. Char_wb es como char, pero no pasa los límites de las palabras. 
Luego se probar distintos analyzers, word mostró mejor desempeño. (Experimentos omitidos por su similitud al experimento 1)

Además, se añade como preprocesador mark_negation, el cual se encarga de darle intencionalidad a palabras neutrales según su contexto (por ejemplo algo positivo como like puede estar en el contexto didn't like y en la bag of words se pierda esta intencionalidad. Entonces, mark_negation lo cambia por like_NEG. Además, se puede ajustar el tamaño de los n-gramas y quitar las stop_words. 
  .

In [0]:
def experimento_1():
    print("Experimento 1")
    TOKENIZER = TweetTokenizer(strip_handles=True)
    vectorizer = CountVectorizer(analyzer='word', preprocessor= mark_negation ,tokenizer= TOKENIZER.tokenize,  ngram_range=(1,4), stop_words='english')
    clf = MultinomialNB()
    text_clf = Pipeline([('vect', vectorizer),('clf',clf)])
    return text_clf

###Experimento 2:
Support Vector Machine:
Se cambia el clasificador por SVC, utilizando un kernel lineal, admitiendo probabilidades.


In [0]:
def experimento_2(): 
    print("Experimento 2")
    # Definimos el vectorizador para convertir el texto a BoW:
    TOKENIZER = TweetTokenizer(strip_handles=True, preserve_case=True )
    vectorizer = CountVectorizer(analyzer='word', preprocessor= mark_negation ,tokenizer= TOKENIZER.tokenize,  ngram_range=(1,4), stop_words='english')  

    # Inicializamos el Clasificador.
    clf = svm.SVC(kernel='linear', probability= True, C=100, gamma='scale')
    # Establecer el pipeline.
    text_clf = Pipeline([('vect', vectorizer), ('clf', clf)])
    return text_clf

### Experimento 3:
Preprocesador de elaboracion propia.
Se probó con classificador NaiveBayes y Logistic regression. Se dejó NaiveBayes por consistencia con los otros experimentos ya que logisticRegression será explicado el en experimento 4 en más detalle. La idea es eliminar las palabras que correspondan a expresiones regulares y que a priori no otorguen información extra al modelo. Como también la aplicación de un stemming para probar las herramientas vistas en el curso.

Como no funcionó, se volvió a mark_negation para los siguientes experimentos.


In [0]:
def experimento_3():
    print("Experimento 3")
    def my_preprocessor(doc):
       preprocessed_doc = re.sub(r'[^\w\s+]', '', str(doc)).lower() 
       doc2 = mark_negation(preprocessed_doc)
       doc3 = porter_stemmer.stem(doc2)
       return doc3
 
    tokenizertweet = TweetTokenizer(strip_handles=True)
    vectorizer = CountVectorizer(analyzer='word', preprocessor = my_preprocessor, tokenizer = tokenizertweet.tokenize, ngram_range=(1,4), stop_words='english')
    clf =  MultinomialNB()
    text_clf = Pipeline([('vect', vectorizer),('clf',clf)])
    return text_clf

###Experimento 4:
Se cambia NaiveBayes por LogisticRegression, con sus parametros por defecto (solver y multi_class, que se escribieron por fines visuales). Además, se aumentó el número de iteraciones a 1000, pues con su valor por defecto (iter=100) no alcanzaba a converger (y por ende mostraba un warning)

In [0]:
def experimento_4():
    print("Experimento 4")
    # Experimento 4: Logistic regression using word n-grams
    tokenizertweet = TweetTokenizer(strip_handles=True)
    vectorizer = CountVectorizer(analyzer="char_wb",tokenizer = tokenizertweet.tokenize, preprocessor = mark_negation, ngram_range=(1,10), stop_words="english")  
    log_mod = LogisticRegression(solver='liblinear',multi_class='ovr', iter=1000)   
    text_clf = Pipeline([('vect', vectorizer), ('clf', log_mod)])
    return text_clf
    
    

###Experimento 5:
En el cuarto experimento, se utilizó la recomendación del cuerpo docente de fijarnos más en la ingeniería de features que en los clasificadores o n-grammas, para ello se decidió usar lexicones desde los <a href="https://affectivetweets.cms.waikato.ac.nz/benchmark/">benchmarks</a> recomendados en el auxiliar 1 y probar su rendimiento. La idea es a partir de datos etiquetados como palabras positivas y negativas desde el conjunto opinion_lexicon, analizar en los tweets cuántas de estas palabras aparecen en los tweets para poder caracterizarlos y después analizar bajo el mismo procedimiento del PipeLine, TweetTokenizator y LogisticRegression el rendimiento del modelo.

In [0]:


class LiuFeatureExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.pos_set = set(opinion_lexicon.positive())
        self.neg_set = set(opinion_lexicon.negative())

    def liu_score(self,sentence): 
        tokenized_sent = self.tokenizer.tokenize(sentence)
        pos_words = 0
        neg_words = 0
        for word in tokenized_sent:
            if word in self.pos_set:
                pos_words += 1
            elif word in self.neg_set:
                neg_words += 1
        return [pos_words,neg_words]

    def transform(self, X, y=None):
        values = []
        for tweet in X:
            values.append(self.liu_score(tweet))

        return(np.array(values))

    def fit(self, X, y=None):
        return self

def experimento_5():
    print("Experimento 5")
    tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True)
    liu_feat = LiuFeatureExtractor(tokenizer)
    vectorizer = CountVectorizer(analyzer='char',tokenizer = tokenizer.tokenize, lowercase=False, preprocessor = mark_negation, ngram_range=(3,5))  
    log_mod = LogisticRegression(solver='liblinear',multi_class='ovr')   
    liu_ngram_clf = Pipeline([ ('feats', 
                            FeatureUnion([ ('ngram', vectorizer), ('liu',liu_feat) ])),
    ('clf', log_mod)])

    return liu_ngram_clf        
      

##Resto del código
Se elige el experimento en la funcion get_classifier


In [104]:
def get_classifier():
  # Se puede elegir cualquiera de los 5, basta con variar el método.
  return experimento_5()

results ={}
def evaulate(predicted, y_test, labels):
    # Importante: al transformar los arreglos de probabilidad a clases,
    # entregar el arreglo de clases aprendido por el clasificador. 
    # (que comunmente, es distinto a ['low', 'medium', 'high'])
    predicted_labels = [labels[np.argmax(item)] for item in predicted]
   
    # Confusion Matrix
    print('Confusion Matrix for {}:\n'.format(key))

    # Classification Report
    print(
        confusion_matrix(y_test,
                         predicted_labels,
                         labels=['low', 'medium', 'high']))

    print('\nClassification Report')
    print(
        classification_report(y_test,
                              predicted_labels,
                              labels=['low', 'medium', 'high']))

    # AUC
    print("auc: ", auc(y_test, predicted))

    # Kappa
    print("kappa:", cohen_kappa_score(y_test, predicted_labels))

    # Accuracy
    print("accuracy:", accuracy_score(y_test, predicted_labels), "\n")

    print('------------------------------------------------------\n\n')
    results[key]={}
    results[key]['auc']=  auc(y_test, predicted)
    results[key]['kappa'] =cohen_kappa_score(y_test, predicted_labels)
    results[key]['accuracy']= accuracy_score(y_test, predicted_labels)
    
def classify(dataset, key):

    X_train, X_test, y_train, y_test = split_dataset(dataset)
    text_clf = get_classifier()
    
    
    
    text_clf.fit(X_train, y_train)
    
    
    # Predecir las probabilidades de intensidad de cada elemento del set de prueba.
    predicted = text_clf.predict_proba(X_test)
    
    # Obtener las clases aprendidas.
    learned_labels = text_clf.classes_

    # Evaluar
    evaulate(predicted, y_test, learned_labels)
    return text_clf, learned_labels
  
classifiers = []
learned_labels_array = []

# Por cada llave en train ('anger', 'fear', 'joy', 'sadness')
for key in nuevotrain:
    classifier, learned_labels = classify(nuevotrain[key], key)
    classifiers.append(classifier)
    learned_labels_array.append(learned_labels)

#print(results)
print('SUMA: ')
print('auc: ' + str(results['anger']['auc'] + results['fear']['auc'] +results['joy']['auc'] + results['sadness']['auc'] ))  
print('kappa: '+ str(results['anger']['kappa'] + results['fear']['kappa'] +results['joy']['kappa'] + results['sadness']['kappa'] )) 
print('accuracy: '+ str(results['anger']['accuracy'] + results['fear']['accuracy'] +results['joy']['accuracy'] + results['sadness']['accuracy'] ))       
    
def predict_target(dataset, classifier, labels):
    # Predecir las probabilidades de intensidad de cada elemento del target set.
    predicted = pd.DataFrame(classifier.predict_proba(dataset.tweet), columns=labels)
    # Agregar ids
    predicted['id'] = dataset.id.values
    # Reordenar
    predicted = predicted[['id', 'low', 'medium', 'high']]
    return predicted
predicted_target = {}

if (not os.path.isdir('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

for idx, key in enumerate(target):
    # Predecir el target set
    predicted_target[key] = predict_target(target[key], classifiers[idx],
                                           learned_labels_array[idx])
    # Guardar predicciones
    predicted_target[key].to_csv('./predictions/{}-pred.txt'.format(key),
                                 sep='\t',
                                 header=False,
                                 index=False)

# Crear archivo zip
a = shutil.make_archive('predictions', 'zip', './predictions')

Experimento 5
Confusion Matrix for anger:

[[21 28 11]
 [12 20 12]
 [ 4 13 25]]

Classification Report
              precision    recall  f1-score   support

         low       0.57      0.35      0.43        60
      medium       0.33      0.45      0.38        44
        high       0.52      0.60      0.56        42

    accuracy                           0.45       146
   macro avg       0.47      0.47      0.46       146
weighted avg       0.48      0.45      0.45       146

auc:  0.4157364148884491
kappa: 0.18866351764378975
accuracy: 0.4520547945205479 

------------------------------------------------------


Experimento 5
Confusion Matrix for fear:

[[49 26  2]
 [21 44 29]
 [ 9 20 52]]

Classification Report
              precision    recall  f1-score   support

         low       0.62      0.64      0.63        77
      medium       0.49      0.47      0.48        94
        high       0.63      0.64      0.63        81

    accuracy                           0.58       252
  

## Comparación métricas  

Para poder comparar los modelos utilizados para los mejores 5 experimentos hechos se realiza un cuadro comparativo donde se muestran los valores de las tres métricas en ellos. 
<hr>
<table class="egt">
  <tr>
    <td>Métricas</td>
    <td>Baseline</td>
    <td>Experimento 1</td>
    <td>Experimento 2</td>
    <td>Experimento 3</td>
    <td>Experimento 4</td>
    <td>Experimento 5</td>
  </tr>
  <tr>
    <td>auc_angry</td>
    <td>0.44 </td>
    <td>0.41</td>
    <td>0.40</td>
    <td>0.40</td>
    <td>0.45</td>
    <td>0.44</td>
  </tr>
  <tr>
    <td>kappa_angry</td>
    <td>0.21 </td>
    <td>0.22</td>
    <td>0.23</td>
    <td>0.23</td>
    <td>0.31</td>
    <td>0.29</td>
  </tr>
  <tr>
    <td>accuracy_angry</td>
    <td>0.7</td>
    <td>0.48</td>
    <td>0.47</td>
    <td>0.49</td>
    <td>0.68</td>
    <td>0.53</td>
  </tr>
  
   <tr>
    <td>auc_fear</td>
    <td>0.44 </td>
    <td>0.41</td>
    <td>0.40</td>
    <td>0.36</td>
    <td>0.45</td>
    <td>0.38</td>
  </tr>
  <tr>
    <td>kappa_fear</td>
    <td>0.19</td>
    <td>0.30</td>
    <td>0.30</td>
    <td>0.37</td>
    <td>0.16</td>
    <td>0.32</td>
  </tr>
  <tr>
    <td>accuracy_fear</td>
    <td>0.58</td>
    <td>0.53</td>
    <td>0.53</td>
    <td>0.58</td>
    <td>0.53</td>
    <td>0.55</td>
  </tr>
  
   <tr>
    <td>auc_joy</td>
    <td>0.40</td>
    <td>0.37</td>
    <td>0.38</td>
    <td>0.38</td>
    <td>0.42</td>
    <td>0.34</td>
  </tr>
  <tr>
    <td>kappa_joy</td>
    <td>0.19</td>
    <td>0.36</td>
    <td>0.35</td>
    <td>0.20</td>
    <td>0.28</td>
    <td>0.32</td>
  </tr>
  <tr>
    <td>accuracy_joy</td>
    <td>0.55</td>
    <td>0.57</td>
    <td>0.57</td>
    <td>0.47</td>
    <td>0.6</td>
    <td>0.55</td>
  </tr>
  
   <tr>
    <td>auc_sadness</td>
    <td>0.45</td>
    <td>0.33</td>
    <td>0.43</td>
    <td>0.43</td>
    <td>0.44</td>
    <td>0.37</td>
  </tr>
  <tr>
    <td>kappa_sadness</td>
    <td>0.11</td>
    <td>0.33</td>
    <td>0.19</td>
    <td>0.36</td>
    <td>0.26</td>
    <td>0.38</td>
  </tr>
  <tr>
    <td>accuracy_sadness</td>
    <td>0.52</td>
    <td>0.55</td>
    <td>0.46</td>
    <td>0.57</td>
    <td>0.55</td>
    <td>0.58</td>
  </tr>
</table>

Analizando esta tabla, se tomaron 4 de estos experimentos para realizar las submissions, donde las mejores dos corresponden las experimento 4 y 5 respectivamente, con los siguientes valores: 

<table class="egt">
  <tr>
    <td>Métricas</td>
    <td>Baseline</td>
    <td>Experimento 4</td>
    <td>Experimento 5</td>
  </tr>
  <tr>
    <td>auc_avg</td>
    <td>0.64</td>
    <td>0.68</td>
    <td>0.68</td>
  </tr>
  <tr>
    <td>kappa_avg</td>
    <td>0.14</td>
    <td>0.25</td>
    <td>0.26</td>
  </tr>
  <tr>
    <td>accuracy_avg</td>
    <td>0.57</td>
    <td>0.58</td>
    <td>0.51</td>
  </tr>
</table>

##Conclusiones


Los resultados mostrados en las tablas fueron realizados se realizaron en base a los últimos experimentos ejecutados, obteniendo los mejores resultados pero cabe mencionar que también se realizaron otro tipo de experimentos como incorporar cross-validation, GridSearchCv, clasificadores como KNN, DecisionTree pero sin buenos resultados.

Cabe destacar que la última optimización fue realizar un UnderSampling del dataset, para balancear las clases de intensidad de sentimientos. 
Cuando se volvieron a correr los experimentos, todos subieron considerablemente su desempeño. Sin embargo, solo la ultima submition a la competencia contiene esta optimización y por ello, existe una gran diferencia de performance, priorizando la métrica kappa por sobre el accuracy.

Al analizar la tabla con los valores obtenidos de las métricas, se puede apreciar que la mayoría de los experimentos superan el baseline entregado por el cuerpo docente.

Se concluye que los experimentos 4 y 5 obtuvieron mejores resultados al momento de generalizar, superando al baseline en la métrica kappa.

Se considera la metrica kappa la más confiable de las tres, dado que es la que define cuanto se aleja el modelo de los resultados aleatorios, por ende, cuando el modelo clasifica algo correctamente, no fue cuestión de azar. Por otro lado, se considera que la metrica menos apropiada (y que fue la peor del modelo) fue la del accuracy, pues al ser la clase medium mucho mayor a las otras, el modelo estará siempre máás inclinado a responder esto. Y como los datos de evualuación también tenian este desbalance, modelos con mayor accuracy podrían elegir medium por cuestión de azar y tener mejores resultados en dicha métrica.

Además, se observa que la restricción del código a técnicas menos sofisticadas como las redes neuronales fue un limitante al momento de obtener mejores resultados en la competencia. 

Las dificultades principales de esta tarea surgen por el desconocimiento de qué features utilizar y en qué momento usar ciertos clasificadores, además de que las métricas utilizadas daban valores relativamente distintos al momento de generalizar el modelo con los datos gold de la competencia.
Este problema mencionado anteriormente puede deberse al desbalance que tienen las clases,como se mencionó anteriormente, generando errores de generalización. 
Por otro lado, experimentando con las diversas técnicas vistas en el curso de Stemming, Lemmatization y Stopwords no mejoraron los resultados por lo que se podría concluir que al momento de analizar tweets es difícil evitar la eliminación de información relevante si se aplican estas técnicas. 

Finalmente, se concluye como mejor resultado el experimento 5, el cual consistía en utilizar LinearRegression, TweetTokenizer y mark_negation. Sin embargo, se considera como la pieza angular del preprocesamiento el balanceo del dataset de entrenamiento, ya que luego de su implementación este experimento y todos los otros aumentaron considerablemente su desempeño general. 
Lamentablemente, existen otras técnicas no experimentadas como embeddings y redes neuronales, entre otras técnicas relacionadas. 
